# تمرین کامپیوتری دوم - قسمت دوم(بازی)
#  نیما مدیرکیاسرایی 810198471

## هدف بازی :
هدف در بازی Sim، مجبور کردن رقیب به ساختن یک مثلث هم رنگ در بین خط های واصل بین نقاط موجود در صفحه است. به عبارتی ، اولین بازیکنی که مجبور شود یک مثلث را به رنگ خود در صفحه بازی کامل کند ، بازنده بازی است.
## تابع hueristic استفاده شده در این الگوریتم :
در کدی که برای پیاده سازی بازی نوشته شده است ، متد evaluate__ برای پیاده سازی تابع hueristic می باشد. در واقع این تابع به نحوی نوشته شده است که به هر عمق بازی که جلو می رود یک ضریب می دهد که آن ضریب 20 قرار داده شده است زیرا قرار است حریف را مجبور به درست کردن یک مثلث هم رنگ کنیم و خودمان نباید آن را درست کنیم بنابراین هرچی عمق جلوتر برود باید امتیاز بیشتری داشته باشد. همچنین در هر دست بازی اگر هر کدام از بازیکنان قرمز یا آبی برنده شود امتیاز آن ها به ترتیب 150 امتیاز زیاد یا کم می شود (چون قرمز در واقع بازیکن maximizing است باید امتیازش بیشتر شود و آبی minimizing است باید امتیازش کم شود). 
## پیاده سازی الگوریتم min-max :

In [22]:
from select import select
# from symbol import dotted_as_name
import turtle
import math
import random
from time import sleep
from sys import argv
import time

MAX, MIN = float('inf'), float('-inf')
global seen_nodes
class Sim:
    # Set true for graphical interface
    GUI = False
    screen = None
    selection = []
    turn = ''
    dots = []
    red = []
    blue = []
    seen_nodes = 0
    available_moves = []
    minimax_depth = 0
    prune = False

    def __init__(self, minimax_depth, prune, gui):
        self.GUI = gui
        self.prune = prune
        self.minimax_depth = minimax_depth
        if self.GUI:
            self.setup_screen()

    def setup_screen(self):
        self.screen = turtle.Screen()
        self.screen.setup(800, 800)
        self.screen.title("Game of SIM")
        self.screen.setworldcoordinates(-1.5, -1.5, 1.5, 1.5)
        self.screen.tracer(0, 0)
        turtle.hideturtle()

    def draw_dot(self, x, y, color):
        turtle.up()
        turtle.goto(x, y)
        turtle.color(color)
        turtle.dot(15)

    def gen_dots(self):
        r = []
        for angle in range(0, 360, 60):
            r.append((math.cos(math.radians(angle)), math.sin(math.radians(angle))))
        return r

    def initialize(self):
        self.selection = []
        self.available_moves = []
        for i in range(0, 6):
            for j in range(i, 6):
                if i != j:
                    self.available_moves.append((i, j))
        if random.randint(0, 2) == 1:
            self.turn = 'red'
        else:
            self.turn = 'blue'
        self.dots = self.gen_dots()
        self.red = []
        self.blue = []
        if self.GUI: turtle.clear()
        self.draw()

    def draw_line(self, p1, p2, color):
        turtle.up()
        turtle.pensize(3)
        turtle.goto(p1)
        turtle.down()
        turtle.color(color)
        turtle.goto(p2)

    def draw_board(self):
        for i in range(len(self.dots)):
            if i in self.selection:
                self.draw_dot(self.dots[i][0], self.dots[i][1], self.turn)
            else:
                self.draw_dot(self.dots[i][0], self.dots[i][1], 'dark gray')

    def draw(self):
        if not self.GUI: return 0
        self.draw_board()
        for i in range(len(self.red)):
            self.draw_line((math.cos(math.radians(self.red[i][0] * 60)), math.sin(math.radians(self.red[i][0] * 60))),
                           (math.cos(math.radians(self.red[i][1] * 60)), math.sin(math.radians(self.red[i][1] * 60))),
                           'red')
        for i in range(len(self.blue)):
            self.draw_line((math.cos(math.radians(self.blue[i][0] * 60)), math.sin(math.radians(self.blue[i][0] * 60))),
                           (math.cos(math.radians(self.blue[i][1] * 60)), math.sin(math.radians(self.blue[i][1] * 60))),
                           'blue')
        self.screen.update()
        sleep(1)
    
                
    def _evaluate(self, depth):
        #TODO
        score = depth * 20
        result = self.gameover(self.red, self.blue)
        if (result == 'red'):
            score = score + 150
            return score
        elif (result == 'blue'):
            score = score - 150
            return score
        else :
            return 'continued'
 

    def minimax(self, depth, player_turn, alpha, beta):
        #TODO
        #Evaluating
        score = depth * 15
        result = self._evaluate(depth)
        if (result != 'continued'):
            return result
        if (depth >= self.minimax_depth):
            return score

        # If this maximizer's move
        if (player_turn == 'red'):
            self.seen_nodes += 1
            best = MIN
            #Traverse all cells
            for i in range(6):
                for j in range(i,6):
                    if (i != j):
                        if (i,j) in self.available_moves:
                            self.red.append((i,j))
                            self.available_moves.remove((i,j))
                            best = max(best, self.minimax(depth + 1, 'blue', alpha, beta))
                            self.available_moves.append((i,j))
                            self.red.remove((i,j))
                            alpha = max(best, alpha)
                            if ((beta <= alpha) and (self.prune == True)):
                                break
                if ((beta <= alpha) and (self.prune == True)):
                    break
            return best
        
        # If this minimizer's move
        else :
            self.seen_nodes += 1
            best = MAX
            #Traverse all cells
            for i in range(6):
                for j in range(i,6):
                    if (i != j):
                        if (i,j) in self.available_moves:                       
                            self.blue.append((i,j))
                            self.available_moves.remove((i,j))
                            best = min(best, self.minimax(depth + 1, 'red', alpha, beta))
                            self.available_moves.append((i,j))
                            self.blue.remove((i,j))
                            beta = min(best, beta)
                            if ((beta <= alpha) and (self.prune == True)):
                                break
                if ((beta <= alpha) and (self.prune == True)):
                    break            
            return best

   # This will return the best possible move for the player 
    def findBestMove(self, depth, player_turn):
        bestval = MIN
        bestmove = ()
        self.seen_nodes += 1
        #Traverse all cells
        for i in range(6):
            for j in range(i,6):
                if (i != j):
                    if (i,j) in self.available_moves:
                        self.red.append((i,j))
                        self.available_moves.remove((i,j))
                        moveval = self.minimax(1, player_turn, MIN, MAX)
                        self.available_moves.append((i,j))
                        self.red.remove((i,j))
                        if (moveval > bestval):
                            bestmove = (i,j)
                            bestval = moveval
        return bestmove

    def enemy(self):
        return random.choice(self.available_moves)

    def _swap_turn(self, turn):
        if turn == 'red':
            return 'blue'
        return 'red'

    def play(self):
        self.initialize()
        while True:
            if self.turn == 'red':
                selection = self.findBestMove(depth=self.minimax_depth, player_turn=self.turn)
                if selection[1] < selection[0]:
                    selection = (selection[1], selection[0])
            else:
                selection = self.enemy()
                if selection[1] < selection[0]:
                    selection = (selection[1], selection[0])
            if selection in self.red or selection in self.blue:
                raise Exception("Duplicate Move!!!")
            if self.turn == 'red':
                self.red.append(selection)
            else:
                self.blue.append(selection)

            self.available_moves.remove(selection)
            self.turn = self._swap_turn(self.turn)
            selection = []
            self.draw()
            r = self.gameover(self.red, self.blue)
            if r != 0:
                return r

    def gameover(self, r, b):
        if len(r) < 3:
            return 0
        r.sort()
        for i in range(len(r) - 2):
            for j in range(i + 1, len(r) - 1):
                for k in range(j + 1, len(r)):
                    if r[i][0] == r[j][0] and r[i][1] == r[k][0] and r[j][1] == r[k][1]:
                        return 'blue'
        if len(b) < 3: return 0
        b.sort()
        for i in range(len(b) - 2):
            for j in range(i + 1, len(b) - 1):
                for k in range(j + 1, len(b)):
                    if b[i][0] == b[j][0] and b[i][1] == b[k][0] and b[j][1] == b[k][1]:
                        return 'red'
        return 0


if __name__=="__main__":
    game = Sim(minimax_depth=7, prune=True, gui=False)
    results = {"red": 0, "blue": 0}
    start = time.process_time()
    for i in range(150):
        print(i)
        results[game.play()] += 1
    end = time.process_time()
    print('Average Elapsed Time : %f' %((end - start)/150)) 
    print('Total Elapsed Time : %f' %((end - start))) 
    print(results)
    print('Average Seen Nodes : %f' %(game.seen_nodes/150))
    print('Winning Chance : %f' %(results['red']/150))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
Average Elapsed Time : 0.380937
Total Elapsed Time : 57.140625
{'red': 145, 'blue': 5}
Average Seen Nodes : 39075.833333
Winning Chance : 0.966667


کد بالا پیاده سازی الگوریتم min-max برای بازی Sim است به نحوی که اگر بخواهیم از تکنیک هرس آلفا بتا استفاده کنیم باید مقدار Prune را برابر True قرار دهیم.

## نتایج اجرا کردن بازی بدون هرس آلفا بتا :
### عمق 1 :
اولین اجرا :    
Average Elapsed Time : 0.000104  

Total Elapsed Time : 0.015625

{'red': 135, 'blue': 15}

Average Seen Nodes : 4.866667

Winning Chance : 0.900000 

دومین اجرا :    
Average Elapsed Time : 0.000208

Total Elapsed Time : 0.031250

{'red': 125, 'blue': 25}

Average Seen Nodes : 5.046667

Winning Chance : 0.833333

سومین اجرا :  
Average Elapsed Time : 0.000104

Total Elapsed Time : 0.015625

{'red': 125, 'blue': 25}

Average Seen Nodes : 4.993333

Winning Chance : 0.833333

### عمق 3 :
اولین اجرا :  
Average Elapsed Time : 0.008542

Total Elapsed Time : 1.281250

{'red': 130, 'blue': 20}

Average Seen Nodes : 475.180000

Winning Chance : 0.866667

دومین اجرا :  
Average Elapsed Time : 0.008646

Total Elapsed Time : 1.296875

{'red': 130, 'blue': 20}

Average Seen Nodes : 473.066667

Winning Chance : 0.866667

سومین اجرا :  
Average Elapsed Time : 0.008750

Total Elapsed Time : 1.312500

{'red': 140, 'blue': 10}

Average Seen Nodes : 477.833333

Winning Chance : 0.933333

### عمق 5 :
اولین اجرا :  
Average Elapsed Time : 0.958333

Total Elapsed Time : 143.750000

{'red': 135, 'blue': 15}

Average Seen Nodes : 44290.880000

Winning Chance : 0.900000

دومین اجرا:  
Average Elapsed Time : 0.985000

Total Elapsed Time : 147.750000

{'red': 132, 'blue': 18}

Average Seen Nodes : 44452.340000

Winning Chance : 0.880000

سومین اجرا :  
Average Elapsed Time : 0.958333

Total Elapsed Time : 143.750000

{'red': 139, 'blue': 11}

Average Seen Nodes : 43809.640000

Winning Chance : 0.926667

## نتایج اجرا کردن بازی همراه با هرس آلفا بتا :
### عمق 1 :
اولین اجرا :  
Average Elapsed Time : 0.000104

Total Elapsed Time : 0.015625

{'red': 129, 'blue': 21}

Average Seen Nodes : 4.913333

Winning Chance : 0.860000

دومین اجرا :   
Average Elapsed Time : 0.000104

Total Elapsed Time : 0.015625

{'red': 117, 'blue': 33}

Average Seen Nodes : 5.140000

Winning Chance : 0.780000

سومین اجرا :  
Average Elapsed Time : 0.000104

Total Elapsed Time : 0.015625

{'red': 136, 'blue': 14}

Average Seen Nodes : 4.820000

Winning Chance : 0.906667

### عمق 3 :
اولین اجرا :  
Average Elapsed Time : 0.002708

Total Elapsed Time : 0.406250

{'red': 127, 'blue': 23}

Average Seen Nodes : 466.253333

Winning Chance : 0.846667

دومین اجرا :  
Average Elapsed Time : 0.002604

Total Elapsed Time : 0.390625

{'red': 133, 'blue': 17}

Average Seen Nodes : 471.373333

Winning Chance : 0.886667

سومین اجرا :  
Average Elapsed Time : 0.002812

Total Elapsed Time : 0.421875

{'red': 132, 'blue': 18}

Average Seen Nodes : 469.600000

Winning Chance : 0.880000

### عمق 5 :
اولین اجرا :  
Average Elapsed Time : 0.038229

Total Elapsed Time : 5.734375

{'red': 135, 'blue': 15}

Average Seen Nodes : 5247.013333

Winning Chance : 0.900000

دومین اجرا :  
Average Elapsed Time : 0.036979

Total Elapsed Time : 5.546875

{'red': 133, 'blue': 17}

Average Seen Nodes : 5204.993333

Winning Chance : 0.886667

سومین اجرا :  
Average Elapsed Time : 0.038437

Total Elapsed Time : 5.765625

{'red': 134, 'blue': 16}

Average Seen Nodes : 5243.006667

Winning Chance : 0.893333

### عمق 7 :
اولین اجرا :  
Average Elapsed Time : 0.386250

Total Elapsed Time : 57.937500

{'red': 143, 'blue': 7}

Average Seen Nodes : 40819.320000

Winning Chance : 0.953333

دومین اجرا :  
Average Elapsed Time : 0.374271

Total Elapsed Time : 56.140625

{'red': 142, 'blue': 8}

Average Seen Nodes : 39325.580000

Winning Chance : 0.946667

سومین اجرا :  
Average Elapsed Time : 0.380937

Total Elapsed Time : 57.140625

{'red': 145, 'blue': 5}

Average Seen Nodes : 39075.833333

Winning Chance : 0.966667

## سوالات  
1) یک تابع heuristic خوب تابعی است که به حالت هایی از بازی که احتمال پیروزی بازیکن ما در آن بیشتر است امتیاز بیشتری بدهد. دلیل انتخاب تابع hueristic که در متد evaluate_ آورده شده است این است که به بعضی از حالت های مناسب امتیاز مختلف می دهد مثلا برای پیروزی بازیکن آبی یا قرمز و بیشتر شدن عمق. اگر روش های دیگری به کار ببریم که این اتفاقات در آن نیفتد ممکن است تایم بیشتری طول بکشد تا به جواب برسیم یا به دقت مناسبی نرسیم.  
2) قطعا رابطه ای وجود دارد. هرچقدر عمق الگوریتم بیشتر باشد، شانس پیروزی بیشتر می شود اما از طرفی مدت زمان اجرا و گره های دیده شده هم افزایش پیدا می کنند.  
3) قطعا ترتیب اضافه کردن فرزندان روی کارآمدی تکنیک هرس کردن تاثیر زیادی دارد. به طوری که در بدترین حالت هیچ شاخه ای هرس نمی شود و باید تمامی گره ها بررسی شوند و در بهترین حالت بتوانیم تعداد حداکثری از شاخه ها را هرس کنیم تا مدت زمان اجرای الگوریتم را کاهش دهیم. در این پیاده سازی من درون دو حله for و به ترتیب نقاط ، فرزندان را اضافه کردم و روی آن ها با استفاده از شرط if شاخه هایی که امکان هرس کردن وجود داشت را هرس کردم.